In [1]:
library(DESeq2)
library(tidyverse)
load("exp.rda")
clin.data <- data.frame(colData(exp))
head(clin.data)


载入需要的程辑包：S4Vectors

载入需要的程辑包：stats4



载入需要的程辑包：BiocGenerics


载入程辑包：'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



载入程辑包：'S4Vectors'


The following object is masked from 'package:utils':

    findMatches


The following objects are masked from 'package:base':

    expand.grid, I, unname


载入需要的程辑包：IRanges


载入程辑包：'IRanges'


The following object is masked from 'package:grDevices':

    windows


载入需要的程辑包：GenomicRanges

载入需要的程辑包：GenomeInfoDb

载入需要的程辑包：SummarizedExperiment

载入需要的程辑包：MatrixGenerics

载入需要的程辑包：matrixStats


In [ ]:
# Select columns
surv.data <- clin.data %>% 
             dplyr::select(barcode,  
                           sample_type, 
                           gender,
                           race,
                           paper_age_at_initial_pathologic_diagnosis, 
                           paper_pathologic_stage, 
                           vital_status, 
                           days_to_last_follow_up, 
                           days_to_death)

head(surv.data)


In [ ]:
# Code for time and status
surv.data <- surv.data %>%      
             mutate(OS.days = case_when(vital_status == "Alive" ~ days_to_last_follow_up,
                                        vital_status == "Dead" ~ days_to_death)) %>%
             mutate(Status = recode(vital_status, "Alive" = 0, "Dead" = 1)) %>% 
             mutate(Status = as.numeric(Status)) %>% 
             mutate(OS.month = round(as.numeric(OS.days)/30, 3))

head(surv.data)


In [ ]:
# Filter for tumor samples and survival data
surv.data <- surv.data %>% 
             filter(sample_type == "Primary Tumor") %>% 
             filter(Status %in% c("0", "1")) %>% 
             filter(OS.month != "NA") 
head(surv.data)


In [ ]:
# Change data types
surv.data <- surv.data %>% 
             mutate(Status = as.numeric(Status)) %>%
             mutate(age = paper_age_at_initial_pathologic_diagnosis) %>% 
             mutate(stage = paper_pathologic_stage) %>% 
             dplyr::select(barcode, OS.month, Status, gender, race, age, stage)

head(surv.data)


In [ ]:
vsd.df <- read.delim("exp_vsd.tsv", stringsAsFactors = F, check.names = F, row.names = 1)

head(vsd.df)


In [ ]:
# Load DEG
deg.df <- read.delim("results_DEGs_DESeq2.tsv")

dim(deg.df)
head(deg.df)


In [ ]:
# Subset rows = gene_id, columns = barcode 
vsd.deg <- vsd.df[deg.df$gene_id, surv.data$barcode]

head(vsd.deg)


In [ ]:
# Transpose rows/columns
vsd.deg <- t(vsd.deg) %>% 
           as.data.frame() 

head(vsd.deg)


In [ ]:
# Check rownames match
any(rownames(surv.data) != rownames(vsd.deg))


In [ ]:
# Bind columns
cox.data <- cbind(surv.data, vsd.deg) 

head(cox.data)


In [ ]:
library(survival)
library(RegParallel)

res <- RegParallel(
  data = cox.data,
  formula = 'Surv(OS.month, Status) ~ [*]',  # [*] placeholder for gene
  FUN = function(formula, data)  
    coxph(formula = formula,
          data = data,
          ties = 'breslow',
          singular.ok = TRUE),
  FUNtype = 'coxph',
  variables = colnames(cox.data)[8:ncol(cox.data)], # only genes
  blocksize = 500,
  cores = 23
)
head(res)


In [ ]:
res.sig <- res %>% 
           filter(P < 0.05) %>% 
           filter(LogRank < 0.05) 

res.sig <- res.sig %>% 
           mutate(gene_id = Term) %>% 
           left_join(deg.df %>% dplyr::select(gene_id, gene_name, log2FoldChange), by= "gene_id") 

head(res.sig)


In [ ]:
res.sig.up <-  res.sig %>% filter(HR > 1) %>% 
               filter(log2FoldChange > 0) %>% 
               arrange(-HR) %>% 
               head(5)

res.sig.up


In [ ]:
res.sig.down <- res.sig %>% 
                filter(HR < 1) %>% 
                filter(log2FoldChange < 0) %>% 
                arrange(HR) %>% 
                head(5)

res.sig.down


In [ ]:
head(cox.data)


In [ ]:
library(survminer)
library(ggfortify)
# Survival curve for top 5 upregulated genes
for (gene in head(res.sig.up$gene_id, 5)) {
    surv_object <- survfit(Surv(OS.month, Status) ~ get(gene), data = cox.data)
    print(ggsurvplot(surv_object, title = paste("Survival Curve for", gene)))
}

# Survival curve for top 5 downregulated genes
for (gene in head(res.sig.down$gene_id, 5)) {
    surv_object <- survfit(Surv(OS.month, Status) ~ get(gene), data = cox.data)
    print(ggsurvplot(surv_object, title = paste("Survival Curve for", gene)))
}